# Seldon V2 Non Kubernetes Local Examples


 * Build if needed and place `seldon` binary in your path
 * Install seldon locally 

In [ ]:
!which seldon

In [ ]:
%env VERBOSE=false

## Model

In [ ]:
!seldon model load -f ./models/sklearn-iris-gs.yaml -v=${VERBOSE}

In [ ]:
!seldon model status --model-name iris -w ModelAvailable -v=${VERBOSE}

In [ ]:
!seldon model infer --model-name iris -v=${VERBOSE} \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon model infer --model-name iris --inference-mode grpc -v=${VERBOSE} \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}'

In [ ]:
!seldon model unload --model-name iris -v=${VERBOSE}

## Experiment

In [ ]:
!seldon model load -f ./experiments/sklearn1.yaml -v=${VERBOSE}
!seldon model load -f ./experiments/sklearn2.yaml -v=${VERBOSE}

In [ ]:
!seldon model status --model-name iris -v=${VERBOSE}
!seldon model status --model-name iris2 -v=${VERBOSE}

In [ ]:
!seldon experiment start -f ./experiments/ab-default-model.yaml -v=${VERBOSE}

In [ ]:
!seldon experiment status -e experiment-sample -w -v=${VERBOSE}

In [ ]:
!seldon model infer --model-name iris -i 50 -v=${VERBOSE} \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!seldon experiment stop -e experiment-sample -v=${VERBOSE}

## Pipeline (WIP)

In [ ]:
!seldon pipeline load -f ./pipelines/example.yaml -v=${VERBOSE}

In [ ]:
!seldon pipeline status -p example -v=${VERBOSE}

In [ ]:
!seldon pipeline unload -p example -v=${VERBOSE}